# **2D (XY) Clustergram** - **RTU** *(ready-to-use)* variant


## Learn more about Clastergram Chart: 
- https://plotly.com/python/clustergram/
- https://dash.plotly.com/dash-bio/clustergram

*To run all cells in this notebook, please use `Run`, then `Run All Cells` options in the top menu. Alternatively, you can press ▶ button above or `ctrl + enter/return` on each cell separately. To modify the content of the cell:*
* *use single-click on the code-like cell*
* *use double-click on the text-like cell*

## 0. Imports

In [1]:
import base64
import io
import os
import flask
import numpy as np
import pandas as pd
import json
from urllib import request

import dash
from dash import html, dcc, dash_table
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
from dash.dash import no_update
import dash_bootstrap_components as dbc
import dash_bio as db

import plotly.express as px

## 1. Back-end python functions

In [2]:
# Generic functions

def generate_font_options(value_list):
    options = []
    for value in value_list:
        options.append({'label':html.Span([value], style={'font-family':value}), 'value':value})
    return options

def generate_color_options(value_list):
    options = []
    for value in value_list:
        options.append({'label':html.Span([value], style={'color':value}), 'value':value})
    return options

def generate_html_label(text):
    return html.Label(children=str(text), style=css_lab1)


In [3]:
# App-specific functions


## 2. Global variables

In [4]:
# Generic variables
COLORS = ['aliceblue', 'antiquewhite', 'aqua', 'aquamarine', 'azure', 'beige', 'bisque', 'black', 'blanchedalmond', 
          'blue', 'blueviolet', 'brown', 'burlywood', 'cadetblue', 'chartreuse', 'chocolate', 'coral', 'cornflowerblue', 
          'cornsilk', 'crimson', 'cyan', 'darkblue', 'darkcyan', 'darkgoldenrod', 'darkgray', 'darkgrey', 'darkgreen', 
          'darkkhaki', 'darkmagenta', 'darkolivegreen', 'darkorange', 'darkorchid', 'darkred', 'darksalmon', 'darkseagreen', 
          'darkslateblue', 'darkslategray', 'darkslategrey', 'darkturquoise', 'darkviolet', 'deeppink', 'deepskyblue', 
          'dimgray', 'dimgrey', 'dodgerblue', 'firebrick', 'floralwhite', 'forestgreen', 'fuchsia', 'gainsboro', 'ghostwhite', 
          'gold', 'goldenrod', 'gray', 'grey', 'green', 'greenyellow', 'honeydew', 'hotpink', 'indianred', 'indigo', 'ivory', 
          'khaki', 'lavender', 'lavenderblush', 'lawngreen', 'lemonchiffon', 'lightblue', 'lightcoral', 'lightcyan', 
          'lightgoldenrodyellow', 'lightgray', 'lightgrey', 'lightgreen', 'lightpink', 'lightsalmon', 'lightseagreen', 
          'lightskyblue', 'lightslategray', 'lightslategrey', 'lightsteelblue', 'lightyellow', 'lime', 'limegreen', 'linen', 
          'magenta', 'maroon', 'mediumaquamarine', 'ediumblue', 'mediumorchid', 'mediumpurple', 'mediumseagreen', 'mediumslateblue', 
          'mediumspringgreen', 'mediumturquoise', 'mediumvioletred', 'midnightblue', 'mintcream', 'mistyrose', 'moccasin', 'navajowhite', 
          'navy', 'oldlace', 'olive', 'olivedrab', 'orange', 'orangered', 'orchid', 'palegoldenrod', 'palegreen', 'paleturquoise', 
          'palevioletred', 'papayawhip', 'peachpuff', 'peru', 'pink', 'plum', 'powderblue', 'purple', 'red', 'rosybrown', 'royalblue', 
          'rebeccapurple', 'saddlebrown', 'salmon', 'sandybrown', 'seagreen', 'seashell', 'sienna', 'silver', 'skyblue', 'slateblue', 
          'slategray', 'slategrey', 'snow', 'springgreen', 'steelblue', 'tan', 'teal', 'thistle', 'tomato', 'turquoise', 'violet', 'wheat', 
          'white', 'whitesmoke', 'yellow', 'yellowgreen']

FONTS = ["Arial", "Balto", "Courier New", "Droid Sans", "Droid Serif", "Droid Sans Mono", "Gravitas One", 
         "Old Standard TT", "Open Sans", "Overpass", "PT Sans Narrow", "Raleway", "Times New Roman"]

TICKS = [{'label': 'outside', 'value': 'outside'}, {'label': 'inside', 'value': 'inside'}, {'label': '(not visible)', 'value': ''}]

In [5]:
# Function-generated variables

FONT_OPTS = generate_font_options(FONTS)
COLOR_OPTS = generate_color_options(COLORS)
CS_OPTS = px.colors.named_colorscales()
CS_SEQ = px.colors.sequential.swatches_continuous().update_layout(width=350)
CS_DIV = px.colors.diverging.swatches_continuous().update_layout(width=350)
CS_CYC = px.colors.cyclical.swatches_continuous().update_layout(width=350)

In [6]:
# tmp variables for input file
#file_path = os.getcwd()       # [string] provide string path to the input file, os.getcwd() takes current path
file_path = "/Users/abadacz/REPOS/GIF/data_graphing/db_clustergram/data"
filename = "input.csv"        # [string] input file name
col_separator = ','           # [string] type of the column separator in the input file: '\t' for tabulator, ' ' for space, ',' for comma, etc.

input_path = file_path
df = pd.read_csv(str(input_path+"/"+filename).replace('//','/'), sep=col_separator)

## 3. Layout components

### A. CSS styles (inline)

In [7]:
color1 = '#D6F2FA'    # light blue *
color2 = '#738fab'    # gray-teal font
color3 = '#008CBA'    # dark teal *
color4 = '#006699'    # darkest teal
color5 = ''    # 
block = {'display':'block'}
inline = {'display':'inline-block'}
drop_h = {'height':'34px'}
fs_s = {'font-size':'12px'}
fs_m = {'font-size':'14px'}
fs_l = {'font-size':'16px'}
fs_x = {'font-size':'20px'}
css_div = {'display':'inline-block'}
css_lpd = {**css_div, 'overflow-y':'auto', 'overflow-x':'hidden', 'scrollbar-color':'red yellow', 'height':'95vh', 'padding':'5px', 'background-color':color1, 'border-right': '1px solid #008CBA'}
css_rpd = {**css_div, 'flex-grow': '1', 'background-color':'ghostwhite'}
css_btn = {'font-size':'20px', 'background-color':color3, 'color':'white', 'border-radius':'8px', 'border':'1px solid #006B88', 'marginBottom':'10px'}
css_lab = {'font-style':'italic', 'display':'block', 'margin-top':'5px'}
css_lab1 = {**css_lab, 'color':color3, 'font-size':'16px'}
css_lab2 = {**css_lab, 'color':color2, 'font-size':'14px'}
css_inp = {'marginBottom':'10px', 'width':'60%', 'display':'inline-block', 'font-size':'14px', 'padding':'6px 0'}
#css_val = {'width':'30%', 'marginBottom':'20px'}
#css_rad = {'padding': '1vh 2.5vw 0 0'}
#css_col = {'display':'inline-block', 'width':'8%', 'padding':'3px 3px 3px 3px', 'font-size':'12px'}

css_upload_box = {'background-color': '#f9f9f9', 'width': '25vw', 'min-height': '60px', 'height': '20vh', 'border-width': '2px', 'border-color':color4, 'border-style': 'dashed', 'border-radius': '5px', 'display': 'table-cell', 'vertical-align': 'middle', 'text-align': 'center'}

### B. Variables

In [8]:
tooltip = {'img-format':'select format of exported static image: png, svg, jpeg, webp, pdf, eps',
           'img-filename':'provide custom filename for exported static image',
           'img-height':'enter the expected height (in px) of exported static image ',
           'img-width':'enter the expected width (in px) of exported static image',
           'img-scale':'enter the expected scale ratio of exported static image ',
           
           'graph-title':'provide custom text for the graph title',
           'title-font':'customize title font: \n1) select font family, \n2) select font color, \n3) select font size',
           'title-position':'customize title position: \n1) graph title horizontal position, \n2) graph title vertical position',
           'graph-size':'provide size of an interactive graph in px: \n1) height, \n2) width',
           'graph-legend':'legend settings: \n1) if True the legend is visible, if False the legend is hidden, \n2) legend X position, \n3) legend Y position',
           'X-title':'provide custom text for the X-axis title',
           'Y-title':'provide custom text for the Y-axis title',
           'margin':'values (in px) for graph margins in order [left, top, right, bottom] \ntype "d" for the defaults',
           'bg-colors':'background colors: \n1) background color in the plottiong area, \n2) background color of the drawing area',
           'X-axis-font':'customize X-axis font: \n1) select font family, \n2) select font color, \n3) select font size',
           'X-line':'X-axis line: \n1) if True the X axis is visible, if False X axis is hidden, \n2) X line width in px, \n3) X line color',
           'X-mirror':'if True the X axis complementary mirror axis is visible [on the top/bottom of the graph]',
           'X-ticks':'X axis ticks (marks): \n1) position of X axis ticks: "outside", "inside", "" (not visible), \n2) color of X axis ticks, \n3) width (in px) of X axis ticks, \n4) length (in px) of X axis ticks',
           'X-tick-labels':'X axis ticks (labels): \n1) if True the labels of X axis ticks are visible, \n2) font color of ticks labels for X axis, \n3) font size of ticks labels for X axis, \n4) side of the graph where the ticks and labels will apear; enter "top" or "bottom", \n5) angle of X axis ticks in range 0 - 360, \n6) list of strings with tick labels for X axis; should match the number of data columns in the input file; if empty list, the keys (column names) from the header will be used',
           'Y-axis-font':'customize Y-axis font: \n1) select font family, \n2) select font color, \n3) select font size',
           'Y-title-pos':'position of the Y-axis title: \n1) Y axis title horizontal position; balow 0 - left side of the graph, above 1 - right side of the graph, \n2) Y axis title vertical position; usually value around 0.5 center title with the heatmap, \n3) the angle of the text: 0 is horizontal; -90 is vertical-left; 90 is vertical-right',
           'Y-line':'Y-axis line: \n1) if True the Y axis is visible, if False Y axis is hidden, \n2) Y line width in px, \n3) Y line color',
           'Y-mirror':'if True the Y axis complementary mirror axis is visible [on the top/bottom of the graph]',
           'Y-ticks':'Y axis ticks (marks): \n1) position of Y axis ticks: "outside", "inside", "" (not visible), \n2) color of Y axis ticks, \n3) width (in px) of Y axis ticks, \n4) length (in px) of Y axis ticks',
           'Y-tick-labels':'Y axis ticks (labels): \n1) if True the labels of Y axis ticks are visible, \n2) font color of ticks labels for Y axis, \n3) font size of ticks labels for Y axis, \n4) side of the graph where the ticks and labels will apear; enter "left" or "right", \n5) angle of Y axis ticks in range 0 - 360',
           'Y-labels-data':'custom Y-axis ticks: \n1) if True, text labels for the Y axis in the heatmap section can be dispalyed; otherwise an automatic numerical (data index) values will be dispalyed; \n2) name of the column in the input used for labeling ticks on the Y-axis for the heatmap section, \n3) number of ticks on the zoomed Y axis with text tick labels, \n4) specify the zoomed fragment along normalized Y axis; 0 - bottom, 1 - top; text labels switches off the interactive Y axis zooming - this variable can zoom the Y axis though;',

           'center-vals':'if True, center the values of the heatmap about zero',
           'display-cutoff':'standardized values below the negative of this value will be colored with one shade, and the values that are above this value will be colored with another',
           'color-scale':'colorscale for the heatmap: \nselect built-in Plotly color scale \n*use button to preview rendering of available color scales)',
           'colorbar':'heatmap colorbar settings: \n1) if True, the colorbar (heatmap colorscale) will be displayed, \n2) vertical length of the colorbar; default = 1 - dendro_ratio[0], means that the colorbar height matches the heatmap height, \n3) horizontal position of the colorbar; value above 1 places colorbar on the right margin of the plotting area, 4) vertical position of the center of the colorbar; default = 0.5*(1-dendro_ratio[0]), means that the colorbar will be aligned vertically with the heatmap',
           'label-name':'custom name displayed on the interactive labels for the heatmap points',
           'label-X':'label for X data displayed on the interactive label; \nby default the ticks of X axis will be used as data; \n"none" will switch off the option',
           'label-Y':'label for Y data displayed on the interactive label; \nby default the ticks of Y axis will be used as data; \n"none" will switch off the option',
           'label-Z':'label for Z data displayed on the interactive label; \nby default the heatmap (Z) values will be used as data; \n"none" will switch off the option',
           'label-custom':'name of the custom label; \nlabel element is any string;',
           'label-dimension':'dimension options are: "x", "y", "z"; \ndimension specifies the data structure;',
           'label-file':'filename with data for custom label; \nselect filename from the list of pre-loaded files; \n load additional files in the inputs section, if needed;',
           'label-data':'data is an array (1D for "x" or "y", 2D for "z") provided explicitly or as a string of the column name from the input file;',
           'label-font-size':'font size of interactive labels',
           'label-align':'horizontal alignment of the text content within on-hover label box; \nselect option: "left", "right", "auto"',
           
           'row-prefix':'custom name of the dendrogram clusters in rows',
           'col-prefix':'custom name of the dendrogram clusters in columns',
           'dendro-line':'line width [row, column] for the dendrograms',
           'dendro-linkage':'maximum linkage value [row, column] for which unique colors are assigned to clusters',
           'dendro-colors':'list of colors to use for different clusters in the dendrogram that have a root under the threshold for each dimension',
           'leaf-order':'determine leaf order that maximizes similarity between neighboring leaves',
           'scale-ratio':'proportions of dendrogram vs. cluster-bars and heatmap vs. dendrogram: \n1) scale ratio between dendrogram and cluster-bar sections, \n2) scale ratio between heatmap and X axis dendrogram, \n3) scale ratio between heatmap and Y axis dendrogram',
           'dendro-labels':'Interactive labels for dendrograms: \n1) whether to list all cluster elements on interactive labels for Y dendrogram, \n2) number of elements names listed per single line on interactive labels (prevents overly wide labels)',

           'export-html':'if True the generated interactive HTML graph will be automatically saved to file',
}

### C. Components (option menu)

In [9]:
## 1. UPLOAD INPUTS
opts_inputs = [
  html.Label('from file system: ', id='custom-label', style=css_lab2),
  dcc.Upload(id='upload-box', filename='', className='upload-box', style=css_upload_box, max_size=-1, multiple=True,
      children=html.Div([html.P('drag & drop', style={'color':color2}), html.P('or click to browse', style={'color':color2})]),
  ),
  html.Label('from online resources: ', id='custom-url-label', style=css_lab2),
  dcc.Input(id="custom-url", type="url", placeholder="enter URL", value='', debounce=False, style={**inline, 'width':'64%'}),
  dbc.Button("download", id="download-btn", n_clicks=0, size="sm", outline=True, color="secondary", className="align-top", style={**drop_h, 'margin':'0 0 0 2%', 'width':'33%'}),
  html.P(''),
  html.Label(id="settings-upload-label", style=css_lab1),
  html.Div(id="settings-upload-inputs", className="mt-3")
]

In [10]:
## 2. ADJUST ANALYSIS SETTINGS

opts_analysis = [

]

In [11]:
## 3. GENERAL GRAPH SETTINGS

graph_advanced_layout = [
  html.Div([
    html.Label('margins:', style={**css_lab2, **inline}, className='col-3 p-0', title=tooltip['margin']),
    html.Div([
      dcc.Input(id="margin-l", type="text", placeholder="left", value='0', debounce=False, style={'width':'25%'}),
      dcc.Input(id="margin-t", type="text", placeholder="top", value='d', debounce=False, style={'width':'25%'}),
      dcc.Input(id="margin-r", type="text", placeholder="right", value='350', debounce=False, style={'width':'25%'}),
      dcc.Input(id="margin-b", type="text", placeholder="bottom", value='d', debounce=False, style={'width':'25%'}),
    ], className='col-9'),    
  ], className="row align-items-center"),
  html.Div([
    html.Label('background:', style={**css_lab2, **inline}, className='col-3 p-0', title=tooltip['bg-colors']),
    html.Div([
      dcc.Dropdown(COLOR_OPTS, id="plotting-c", placeholder="plotting bgc", style={**inline, **drop_h, 'width':'50%'}),
      dcc.Dropdown(COLOR_OPTS, id="drawing-c", placeholder="drawing bgc", style={**inline, **drop_h, 'width':'50%'}),
    ], className='col-9 mt-2'),
  ], className="row align-items-center"),
]

graph_advanced_title = [
  html.Div([
    html.Label('title font:', style={**css_lab2, **inline}, className='col-3 p-0', title=tooltip['title-font']),
    html.Div([
      dcc.Dropdown(FONT_OPTS, id="title-font", multi=False, placeholder="font", className="wide-opts", style={**inline, **drop_h,'width':'33%'}),
      dcc.Dropdown(COLOR_OPTS, id="title-color", multi=False, placeholder="color", className="wide-opts", style={**inline, **drop_h, 'width':'33%'}),
      dcc.Input(id="title-size", type="number", min=1, placeholder="size", value=24, debounce=True, style={'width':'33%', 'vertical-align': 'top'}),
    ], className='col-9'),
  ], className="row align-items-center"),
  html.Div([
    html.Label('title position:', style={**css_lab2, **inline}, className='col-3 p-0', title=tooltip['title-position']),
    html.Div([
      dcc.Input(id="title-posX", type="number", min=0, step=0.01, placeholder="pos X", value=0.38, debounce=True, style={'width':'50%'}),
      dcc.Input(id="title-posY", type="number", min=0, step=0.01, placeholder="pos Y", value=0.9, debounce=True, style={'width':'50%'}),
    ], className='col-9 mt-1'),
  ], className="row align-items-center"),
]

graph_advanced_X = [
  html.Div([
    html.Label('X title font:', style={**css_lab2, **inline}, className='col-3 p-0', title=tooltip['X-axis-font']),
    html.Div([
      dcc.Dropdown(FONT_OPTS, id="X-axis-font", placeholder="font", className="wide-opts", style={**inline, **drop_h,'width':'33%'}),
      dcc.Dropdown(COLOR_OPTS, id="X-axis-color", placeholder="color", className="wide-opts", style={**inline, **drop_h, 'width':'33%'}),
      dcc.Input(id="X-axis-size", type="number", min=1, placeholder="size", value=20, debounce=True, style={'width':'33%', 'vertical-align': 'top'}),
    ], className='col-9'),
  ], className="row align-items-center"),
  html.Div([
    html.Label('X line: ', style={**css_lab2, **inline}, className='col-3 p-0', title=tooltip['X-line']),
    html.Div([
      dcc.Checklist(id='X-line', options=[{'label': 'YES', 'value': 'True'}], value=['True'], inline=True, style={**inline, 'width':'33%', 'vertical-align': 'super'}),
      dcc.Dropdown(COLOR_OPTS, id="X-line-color", placeholder="color", className="wide-opts", style={**inline, **drop_h, 'width':'33%'}),
      dcc.Input(id="X-line-width", type="number", placeholder="width", min=0, value=1, step=0.1, debounce=True, style={'width':'33%', 'vertical-align': 'top'}),
    ], className='col-9 mt-2',),
  ], className="row align-items-center"),
  html.Div([
    html.Label('X mirror: ', style={**css_lab2, **inline}, className='col-3 p-0', title=tooltip['X-mirror']),
    html.Div([
      dcc.Checklist(id='X-mirror', options=[{'label': 'YES', 'value': 'True'}], inline=True, style={**inline, 'width':'34%'}),
    ], className='col-9 mt-2',),
  ], className="row align-items-center"),
  html.Div([
    html.Label('X ticks: ', style={**css_lab2, **inline}, className='col-3 p-0', title=tooltip['X-ticks']),
    html.Div([
      dcc.Dropdown(TICKS, id="X-ticks", placeholder="type", className="wide-opts", style={**inline, **drop_h, 'width':'33%'}),
      dcc.Dropdown(COLOR_OPTS, id="X-ticks-color", placeholder="color", className="wide-opts", style={**inline, **drop_h, 'width':'33%'}),
      html.Br(),
      dcc.Input(id="X-ticks-width", type="number", placeholder="width", min=0, value=1, step=0.1, debounce=True, style={'width':'33%', 'vertical-align': 'top'}),
      dcc.Input(id="X-ticks-len", type="number", placeholder="length", min=0, value=5, step=0.1, debounce=True, style={'width':'33%', 'vertical-align': 'top'}),
    ], className='col-9 mt-2',),
  ], className="row align-items-center"),
  html.Div([
    html.Label('tick labels: ', style={**css_lab2, **inline}, className='col-3 p-0', title=tooltip['X-tick-labels']),
    html.Div([
      dcc.Checklist(id='X-tick-labels', options=[{'label': 'YES', 'value': 'True'}], value=['True'], inline=True, style={**inline, 'width':'33%', 'vertical-align': 'super'}),
      dcc.Dropdown(COLOR_OPTS, id="X-tick-font-color", placeholder="color", className="wide-opts", style={**inline, **drop_h, 'width':'33%'}),
      dcc.Input(id="X-tick-font-size", type="number", placeholder="width", min=1, value=14, step=1, debounce=True, style={'width':'33%', 'vertical-align': 'top'}),
      html.Br(),
      dcc.Dropdown(['top', 'bottom'], id="X-tick-font-pos", placeholder="position", className="wide-opts", style={**inline, **drop_h, 'width':'33%'}),
      dcc.Input(id="X-tick-font-angle", type="number", placeholder="width", min=0, max=360, value=45, step=1, debounce=True, style={'width':'33%', 'vertical-align': 'top'}),
      html.Br(),
      dcc.Input(id="X-tick-labels-list", type="text", placeholder="comma-separated custom X-tick labels", debounce=False, style={'width':'100%'}),
    ], className='col-9 mt-2',),
  ], className="row align-items-center"),
]

graph_advanced_Y = [
  html.Div([
    html.Label('Y title font:', style={**css_lab2, **inline}, className='col-3 p-0', title=tooltip['Y-axis-font']),
    html.Div([
      dcc.Dropdown(FONT_OPTS, id="Y-axis-font", placeholder="font", className="wide-opts", style={**inline, **drop_h,'width':'33%'}),
      dcc.Dropdown(COLOR_OPTS, id="Y-axis-color", placeholder="color", className="wide-opts", style={**inline, **drop_h, 'width':'33%'}),
      dcc.Input(id="Y-axis-size", type="number", min=1, placeholder="size", value=20, debounce=True, style={'width':'33%', 'vertical-align': 'top'}),
    ], className='col-9'),
  ], className="row align-items-center"),
  html.Div([
    html.Label('Y title pos:', style={**css_lab2, **inline}, className='col-3 p-0', title=tooltip['Y-title-pos']),
    html.Div([
      dcc.Input(id="Y-axis-posX", type="number", placeholder="pos X", value=1.24, debounce=True, style={'width':'33%'}),
      dcc.Input(id="Y-axis-posY", type="number", placeholder="pos Y", value=0.42, debounce=True, style={'width':'33%'}),
      dcc.Input(id="Y-axis-angle", type="number", placeholder="angle", value=90, debounce=True, style={'width':'33%'}),
    ], className='col-9'),
  ], className="row align-items-center"),
  html.Div([
    html.Label('Y axis line: ', style={**css_lab2, **inline}, className='col-3 p-0', title=tooltip['Y-line']),
    html.Div([
      dcc.Checklist(id='Y-line', options=[{'label': 'YES', 'value': 'True'}], value=['True'], inline=True, style={**inline, 'width':'33%', 'vertical-align': 'super'}),
      dcc.Dropdown(COLOR_OPTS, id="Y-line-color", placeholder="color", className="wide-opts", style={**inline, **drop_h, 'width':'33%'}),
      dcc.Input(id="Y-line-width", type="number", placeholder="width", min=0, value=1, step=0.1, debounce=True, style={'width':'33%', 'vertical-align': 'top'}),
    ], className='col-9 mt-2',),
  ], className="row align-items-center"),
  html.Div([
    html.Label('Y mirror: ', style={**css_lab2, **inline}, className='col-3 p-0', title=tooltip['Y-mirror']),
    html.Div([
      dcc.Checklist(id='Y-mirror', options=[{'label': 'YES', 'value': 'True'}], inline=True, style={**inline, 'width':'34%'}),
    ], className='col-9 mt-2',),
  ], className="row align-items-center"),
  html.Div([
    html.Label('Y ticks: ', style={**css_lab2, **inline}, className='col-3 p-0', title=tooltip['Y-ticks']),
    html.Div([
      dcc.Dropdown(TICKS, id="Y-ticks", placeholder="type", className="wide-opts", style={**inline, **drop_h, 'width':'33%'}),
      dcc.Dropdown(COLOR_OPTS, id="Y-ticks-color", placeholder="color", className="wide-opts", style={**inline, **drop_h, 'width':'33%'}),
      html.Br(),
      dcc.Input(id="Y-ticks-width", type="number", placeholder="width", min=0, value=1, step=0.1, debounce=True, style={'width':'33%', 'vertical-align': 'top'}),
      dcc.Input(id="Y-ticks-len", type="number", placeholder="length", min=0, value=5, step=0.1, debounce=True, style={'width':'33%', 'vertical-align': 'top'}),
    ], className='col-9 mt-2',),
  ], className="row align-items-center"),
  html.Div([
    html.Label('tick labels: ', style={**css_lab2, **inline}, className='col-3 p-0', title=tooltip['Y-tick-labels']),
    html.Div([
      dcc.Checklist(id='Y-tick-labels', options=[{'label': 'YES', 'value': 'True'}], value=['True'], inline=True, style={**inline, 'width':'33%', 'vertical-align': 'super'}),
      dcc.Dropdown(COLOR_OPTS, id="Y-tick-font-color", placeholder="color", className="wide-opts", style={**inline, **drop_h, 'width':'33%'}),
      dcc.Input(id="Y-tick-font-size", type="number", placeholder="width", min=1, value=14, step=1, debounce=True, style={'width':'33%', 'vertical-align': 'top'}),
      html.Br(),
      dcc.Dropdown(['left', 'right'], id="Y-tick-font-pos", placeholder="position", className="wide-opts", style={**inline, **drop_h, 'width':'33%'}),
      dcc.Input(id="Y-tick-font-angle", type="number", placeholder="width", min=0, max=360, value=0, step=1, debounce=True, style={'width':'33%', 'vertical-align': 'top'}),
    ], className='col-9 mt-2',),
  ], className="row align-items-center"),
  html.Div([
    html.Label('data labels: ', style={**css_lab2, **inline}, className='col-3 p-0', title=tooltip['Y-labels-data']),
    html.Div([
      dcc.Checklist(id='Y-labels-data', options=[{'label': 'YES', 'value': 'True'}], value=[], inline=True, style={**inline, 'width':'33%', 'vertical-align': 'super'}),
      dcc.Dropdown([], id="Y-labels-col", placeholder="data col", className="wide-opts", style={**inline, **drop_h, 'width':'33%'}),
      html.Br(),
      dcc.Input(id="Y-labels-number", type="number", placeholder="ticks number", min=1, value=10, step=1, debounce=True, style={'width':'33%'}),
      html.Div([
        dcc.Input(id="Y-labels-zoom-from", type="number", placeholder="zoom from", min=0, max=1, step=0.01, debounce=True, style={'width':'50%'}),
        dcc.Input(id="Y-labels-zoom-to", type="number", placeholder="zoom to", min=0, max=1, step=0.01, debounce=True, style={'width':'50%'}),
      ], style={**inline,'width':'66%', 'border':'1px solid #738fab', 'border-radius': '0.25rem'}),
    ], className='col-9 mt-2',),
  ], className="row align-items-center"),
]

opts_graph = [
  html.Div([
    html.Label('graph title: ', style={**css_lab2, **inline}, className='col-3', title=tooltip['graph-title']),
    html.Div([
      dcc.Input(id="graph-title", type="text", placeholder="enter graph title", value='', debounce=False, style={'width':'100%'}),
    ], className='col-9',),
  ], className="row align-items-center"),
  html.Div([
    html.Label('graph size:', style={**css_lab2, **inline}, className='col-3', title=tooltip['graph-size']),
    html.Div([
      dcc.Input(id="graph-height", type="number", min=100, placeholder="height", value=600, debounce=True, style={'width':'50%'}),
      dcc.Input(id="graph-width", type="number", min=100, placeholder="width", value=800, debounce=True, style={'width':'50%'}),
    ], className='col-9 mt-2'),
  ], className="row align-items-center"),
  html.Div([
    html.Label('X-axis title: ', style={**css_lab2, **inline}, className='col-3', title=tooltip['X-title']),
    html.Div([
      dcc.Input(id="X-title", type="text", placeholder="enter X-axis title", value='', debounce=False, style={'width':'100%'}),
    ], className='col-9 mt-2',),
  ], className="row align-items-center"),
  html.Div([
    html.Label('Y-axis title: ', style={**css_lab2, **inline}, className='col-3', title=tooltip['Y-title']),
    html.Div([
      dcc.Input(id="Y-title", type="text", placeholder="enter Y-axis title", value='', debounce=False, style={'width':'100%'}),
    ], className='col-9 mt-2',),
  ], className="row align-items-center"),
  html.Div([
    html.Label('legend: ', style={**css_lab2, **inline}, className='col-3', title=tooltip['graph-legend']),
    html.Div([
      dcc.Checklist(id='graph-legend', options=[{'label': 'YES', 'value': 'True'}], value=['True'], inline=True, style={**inline, 'width':'34%'}),
      dcc.Input(id="legend-X", type="number", placeholder="X", value=1.04, step=0.01, debounce=True, style={'width':'33%'}),
      dcc.Input(id="legend-Y", type="number", placeholder="Y", value=1.00, step=0.01, debounce=True, style={'width':'33%'}),
    ], className='col-9 mt-2',),
  ], className="row align-items-center"),

  html.Div([
    dbc.Accordion([
      dbc.AccordionItem(graph_advanced_layout, title="ADVANCED LAYOUT", item_id="graph-1"),
      dbc.AccordionItem(graph_advanced_title, title="ADVANCED TITLE", item_id="graph-2"),
      dbc.AccordionItem(graph_advanced_X, title="ADVANCED X-AXIS", item_id="graph-3"),
      dbc.AccordionItem(graph_advanced_Y, title="ADVANCED Y-AXIS", item_id="graph-4"),
    ], id="graph-advanced", class_name='accordion2', start_collapsed=True, always_open=True, flush=False, style={'width':'100%', 'padding':'0'}),
  ], className="row align-items-center mt-3"), 
]

In [12]:
## A. CUSTOMIZE HEATMAP

modal_cs = html.Div([
  dbc.Button("preview CS", id="modal-cs-btn-open", n_clicks=0, size="sm", outline=True, color="secondary", className="me-1 align-top", style={**drop_h, 'marginRight':'0.5rem', 'width':'85%'}),
  dbc.Modal([
    dbc.ModalHeader(dbc.ModalTitle("95 Built-in color scales: sequential (66), diverging (22), and cyclic (7)"), style={'background-color':color1}),
    dbc.ModalBody(html.Div([
        html.Div([dcc.Graph(figure=CS_SEQ)], style={**inline, 'width':'33%'}), 
        html.Div([dcc.Graph(figure=CS_DIV)], style={**inline, 'width':'33%', 'position':'absolute', 'top':'2.5%', 'left':'30%'}), 
        html.Div([dcc.Graph(figure=CS_CYC)], style={**inline, 'width':'33%', 'position':'absolute', 'top':'2.5%', 'left':'58.5%'})
    ])),
    dbc.ModalFooter(dbc.Button("Close", id="modal-cs-btn-close", n_clicks=0, outline=True, color="secondary", className="me-1 align-top")),
  ], id="modal-cs", size="xl", centered=True, is_open=False, scrollable=True),
], style={**inline, 'width':'50%', 'vertical-align':'top'})

heatmap_advanced_colorbar =  html.Div([
    html.Label('colorbar: ', className='col-4', title=tooltip['colorbar'], style={**css_lab2, **inline}),
    html.Div([
      dcc.Checklist(id='colorbar', options=[{'label': 'YES', 'value': 'True'}], value=['True'], inline=True, style={**inline, 'width':'50%'}),
      dcc.Input(id="cb-length", type="number", min=0, placeholder="length", debounce=True, style={'width':'50%'}),
      html.Br(),
      dcc.Input(id="cb-X", type="number", placeholder="X pos", debounce=True, className='mt-2', style={'width':'50%'}),
      dcc.Input(id="cb-Y", type="number", placeholder="Y pos", debounce=True, className='mt-2', style={'width':'50%'}),
    ], className='col-8 mt-2',),
  ], className="row align-items-center"),

heatmap_advanced_labels = [
  html.Div([
    html.Label('label ID: ', style={**css_lab2, **inline}, className='col-4', title=tooltip['label-name']),
    html.Div([
      dcc.Input(id="label-name", type="text", placeholder="label name", debounce=True, style={'width':'100%'}),
    ], className='col-8',),
  ], className="row align-items-center"),
  html.Div([
    html.Label('label X: ', style={**css_lab2, **inline}, className='col-4', title=tooltip['label-X']),
    html.Div([
      dcc.Input(id="label-X", type="text", placeholder="label X", debounce=True, style={'width':'100%'}),
    ], className='col-8 mt-2',),
  ], className="row align-items-center"),
  html.Div([
    html.Label('label Y: ', style={**css_lab2, **inline}, className='col-4', title=tooltip['label-Y']),
    html.Div([
      dcc.Input(id="label-Y", type="text", placeholder="label Y", debounce=True, style={'width':'100%'}),
    ], className='col-8 mt-2',),
  ], className="row align-items-center"),
  html.Div([
    html.Label('label Z: ', style={**css_lab2, **inline}, className='col-4', title=tooltip['label-Z']),
    html.Div([
      dcc.Input(id="label-Z", type="text", placeholder="label Z", debounce=True, style={'width':'100%'}),
    ], className='col-8 mt-2',),
  ], className="row align-items-center"),
  html.Hr(),
  html.Div([
    html.Label('label size: ', style={**css_lab2, **inline}, className='col-4', title=tooltip['label-font-size']),
    html.Div([
      dcc.Input(id="label-font-size", type="text", placeholder="font size", debounce=True, style={'width':'50%'}),
    ], className='col-8 mt-2',),
  ], className="row align-items-center"),
  html.Div([
    html.Label('label align: ', style={**css_lab2, **inline}, className='col-4', title=tooltip['label-align']),
    html.Div([
      dcc.Dropdown(['left', 'right', 'auto'], id="label-align", placeholder="align text", style={**inline, **drop_h, 'width':'50%'}),
    ], className='col-8 mt-2',),
  ], className="row align-items-center"), 
  html.Hr(),
  html.Label('Add custom labels', style={**css_lab2, **inline}, className='col-12'),
  html.Div([
    html.Label('name: ', style={**css_lab2, **inline}, className='col-4', title=tooltip['label-custom']),
    html.Div([
      dcc.Input(id="label-custom", type="text", placeholder="custom label", debounce=True, style={'width':'100%'}),
    ], className='col-8 mt-2',),
  ], className="row align-items-center"),
  html.Div([
    html.Label('dimension: ', style={**css_lab2, **inline}, className='col-4', title=tooltip['label-dimension']),
    html.Div([
      dcc.RadioItems(['X', 'Y', 'Z'], id='label-dimension', value='True', inline=True, labelStyle={'padding-right':'1.5vw'}),
    ], className='col-8 mt-2',),
  ], className="row align-items-center"),
  html.Div([
    html.Label('filename: ', className='col-4', title=tooltip['label-file'], style={**css_lab2, **inline, 'vertical-align':'top'}),
    html.Div([
      dcc.Dropdown([], id="label-file", placeholder="label file", style={**inline, **drop_h, 'width':'100%'}),
    ], className='col-8 mt-2',),
  ], className="row align-items-center"),
  html.Div([
    html.Label('data: ', style={**css_lab2, **inline}, className='col-4', title=tooltip['label-data']),
    html.Div([
      dcc.Input(id="label-data", type="text", placeholder="label data", debounce=True, style={'width':'100%'}),
    ], className='col-8 mt-2',),
  ], className="row align-items-center"),
]


opts_heatmap = [
  html.Div([
    html.Label('center values: ', className='col-4', title=tooltip['center-vals'], style={**css_lab2, **inline}),
    html.Div([
      dcc.Checklist(id='center-vals', options=[{'label': 'YES', 'value': 'True'}], value=['True'], inline=True, style={**inline, 'width':'37.5%'}),
    ], className='col-8',),
  ], className="row align-items-center"),
  html.Div([
    html.Label('display cutoff: ', style={**css_lab2, **inline}, className='col-4', title=tooltip['display-cutoff']),
    html.Div([
      dcc.Input(id="display-cutoff", type="number", placeholder="cutoff", step=0.01, debounce=True, style={'width':'50%'}),
    ], className='col-8 mt-3',),
  ], className="row align-items-center"),
  html.Div([
    html.Label('color scale: ', className='col-4', title=tooltip['color-scale'], style={**css_lab2, **inline, 'vertical-align':'top'}),
    html.Div([
      modal_cs,
      dcc.Dropdown(CS_OPTS, id="color-scale", placeholder="color scale", style={**inline, **drop_h, 'width':'50%'}),
    ], className='col-8 mt-2',),
  ], className="row align-items-center"),
    
  html.Div([
    dbc.Accordion([
      dbc.AccordionItem(heatmap_advanced_colorbar, title="COLORBAR", item_id="heatmap-1"),
      dbc.AccordionItem(heatmap_advanced_labels, title="INTERACTIVE LABELS", item_id="heatmap-2"),
    ], id="heatmap-advanced", class_name='accordion2', start_collapsed=True, always_open=True, flush=False, style={'width':'100%', 'padding':'0'}),
  ], className="row align-items-center mt-3"), 
]    


In [13]:
## B. CUSTOMIZE DENDROGRAMS [row_prefix, col_prefix, leaf_order, dendro_factor, dendro_ratioX, dendro_ratioY, dendro_threshold, dendro_colors, dendro_line_width, list_all_row, n_per_line_row]

dendro_advanced = [
  html.Div([
    html.Label('leaf order: ', style={**css_lab2, **inline}, className='col-3 p-0', title=tooltip['leaf-order']),
    dcc.RadioItems(options={'True':'YES', 'False':'NO'}, value='True', inline=True, className='col-9', labelStyle={'padding-right':'2.5vw'}),
  ], className="row align-items-center"),

  # scale-ratio
  html.Div([
    html.Label('scale ratio: ', style={**css_lab2, **inline}, className='col-3 p-0', title=tooltip['scale-ratio']),
    html.Div([
      dcc.Input(id="ratio-db", type="number", min=0.1, placeholder="bars", value=0.9, debounce=True, style={'width':'33.3%'}),
      dcc.Input(id="ratio-hdX", type="number", min=0.1, placeholder="dendro-X", value=0.15, debounce=True, style={'width':'33.4%'}),
      dcc.Input(id="ratio-hdY", type="number", min=0.1, placeholder="dendro-Y", value=0.3, debounce=True, style={'width':'33.3%'}),
    ], className='col-9 mt-2',),
  ], className="row align-items-center"),
    
  html.Div([
    html.Label('linkage:', style={**css_lab2, **inline}, className='col-3 p-0', title=tooltip['dendro-linkage']),
    html.Div([
      dcc.Input(id="linkage-r", type="number", min=1, placeholder="row", value=3, debounce=True, style={'width':'50%'}),
      dcc.Input(id="linkage-c", type="number", min=1, placeholder="column", value=3, debounce=True, style={'width':'50%'}),
    ], className='col-9 mt-2'),
  ], className="row align-items-center"),
  html.Div([
    html.Label('linkage colors:', style={**css_lab2, **inline}, className='col-3 p-0', title=tooltip['dendro-colors']),
    html.Div([
      dcc.Dropdown(COLOR_OPTS, id="colors-r", multi=True, placeholder="select n=linkage", style={**inline, **drop_h, 'width':'50%'}),
      dcc.Dropdown(COLOR_OPTS, id="colors-c", multi=True, placeholder="select n=linkage", style={**inline, **drop_h, 'width':'50%'}),
    ], className='col-9 mt-2'),
  ], className="row align-items-center"),
  html.Div([
    html.Label('line width:', style={**css_lab2, **inline}, className='col-3 p-0', title=tooltip['dendro-line']),
    html.Div([
      dcc.Input(id="line-row", type="number", min=0.1, max=10, placeholder="row", value=2, debounce=True, style={'width':'50%'}),
      dcc.Input(id="line-col", type="number", min=0.1, max=10, placeholder="column", value=2, debounce=True, style={'width':'50%'}),
    ], className='col-9 mt-0'),
  ], className="row align-items-center"),
  html.Div([
    html.Label('labels: ', style={**css_lab2, **inline}, className='col-3 p-0', title=tooltip['dendro-labels']),
    html.Div([
      dcc.Dropdown(['True', 'False'], id="dendro-lab", value='True', placeholder="show clusters", style={**inline, **drop_h, 'width':'50%'}),
      dcc.Input(id="dendro-lab-items", type="number", min=1, placeholder="items per line", value=5, debounce=True, style={'width':'50%', 'vertical-align': 'top'}),
    ], className='col-9 mt-2',),    #style={'padding-right':'0.25rem', 'padding-left':'0.3rem'}
  ], className="row align-items-center"),
]

opts_dendro = [
  html.Div([
    html.Label('row prefix: ', style={**css_lab2, **inline}, className='col-3', title=tooltip['row-prefix']),
    html.Div([
      dcc.Input(id="row-name", type="text", placeholder="enter custom row prefix", value='', debounce=False, style={'width':'100%'}),
    ], className='col-9'),
  ], className="row align-items-center"),
  html.Div([
    html.Label('col prefix: ', style={**css_lab2, **inline}, className='col-3', title=tooltip['col-prefix']),
    html.Div([
      dcc.Input(id="col-name", type="text", placeholder="enter custom col prefix", value='', debounce=False, style={'width':'100%'}),
    ], className='col-9 mt-2'),
  ], className="row align-items-center"),

  html.Div([
    dbc.Accordion([
      dbc.AccordionItem(dendro_advanced, title="ADVANCED OPTIONS:", item_id="d-1"),
    ], id="dendro-advanced", class_name='accordion2', start_collapsed=True, always_open=True, flush=False, style={'width':'100%', 'padding':'0'}),
  ], className="row align-items-center mt-3"), 
]

In [14]:
## C. CUSTOMIZE CLUSTER BARS

opts_bars = [

]

In [15]:
## D. EXPORT GRAPH IMAGE [static_img_format, static_img_filename, static_img_height, static_img_width, static_img_scale]
opts_config = [
  html.Div([
    html.Label('format: ', style={**css_lab2, **inline}, className='col-3', title=tooltip['img-format']),
    dcc.Dropdown(id='img-format', placeholder="select IMG format", clearable=False, style={**inline,'width': '75%'}, className='col-12',
        options=[{'label': i, 'value': i} for i in ['png', 'svg', 'jpeg', 'webp', 'pdf', 'eps']], value='svg'),
  ], className="row align-items-center"),
  html.Div([
    html.Label('filename: ', style={**css_lab2, **inline}, className='col-3', title=tooltip['img-filename']),
    html.Div([
      dcc.Input(id="img-name", type="text", placeholder="enter IMG filename", value='clustergram', debounce=False, style={'width':'100%'}),
    ], className='col-9', style={'margin-top':'0.15rem'}),
  ], className="row align-items-center"),
  html.Div([
    html.Label('height [px]:', style={**css_lab2, **inline}, className='col-3', title=tooltip['img-height']),
    html.Div([
      dcc.Input(id="img-height", type="number", min=1200, placeholder="height", value=1200, debounce=True, style={'width':'33%'}),
    ], className='col-9 mt-2'),
  ], className="row align-items-center"),
  html.Div([
    html.Label('width [px]:', style={**css_lab2, **inline}, className='col-3', title=tooltip['img-width']),
    html.Div([
      dcc.Input(id="img-width", type="number", min=800, placeholder="width", value=800, debounce=True, style={'width':'33%'}),
    ], className='col-9 mt-2'),
  ], className="row align-items-center"),
  html.Div([
    html.Label('scale ratio:', style={**css_lab2, **inline}, className='col-3', title=tooltip['img-scale']),
    html.Div([
      dcc.Input(id="img-scale", type="number", min=0.1, placeholder="scale", value=1, debounce=True, style={'width':'33%'}),
    ], className='col-9 mt-2'),
  ], className="row align-items-center"),
  html.Div([
    html.Label('HTML export: ', style={**css_lab2, **inline}, className='col-3', title=tooltip['export-html']),
    html.Div([
      dcc.Checklist(id='export-html', options=[{'label': 'YES', 'value': 'True'}], value=['True'], inline=True, style={**inline, 'width':'33%'}),
      dcc.Input(id="html-name", type="text", placeholder="enter HTML filename", value='clustergram', debounce=False, style={'width':'67%'}),
    ], className='col-9 mt-2',),
  ], className="row align-items-center"),
]

### D. Components assembly

In [16]:
opts = html.Div([
  dbc.Accordion([
    dbc.AccordionItem(opts_inputs, title="1. UPLOAD INPUTS", item_id="item-1"),
    dbc.AccordionItem(opts_analysis, title="2. ADJUST ANALYSIS SETTINGS", item_id="item-2"),
    dbc.AccordionItem(opts_graph, title="3. GENERAL GRAPH SETTINGS", item_id="item-3"),
    dbc.AccordionItem(opts_heatmap, title="A. CUSTOMIZE HEATMAP", item_id="item-4"),
    dbc.AccordionItem(opts_dendro, title="B. CUSTOMIZE DENDROGRAMS", item_id="item-5"),
    dbc.AccordionItem(opts_bars, title="C. CUSTOMIZE CLUSTER BARS", item_id="item-6"),
    dbc.AccordionItem(opts_config, title="D. EXPORT GRAPH", item_id="item-7", className=".container"),
  ], id="accordion", start_collapsed=True, always_open=True, flush=False, style={'width':'25vw'}),
  html.Div(id="accordion-contents", className="mt-3"),
], id='optionsDiv')

## 4. Graph components

## 5. App configuration and final layout

In [17]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP, '/static/mycss.css'])

app.scripts.config.serve_locally=True
app.css.config.serve_locally=True

app.layout = html.Div([

  # STORAGE
  dcc.Store(id="user-files-list", data={}, storage_type='session'),
  dcc.Store(id="graph-data", data={}, storage_type='session'),

  # OPTIONS
  html.Div([
    html.Button('Show Options', id='options', n_clicks=0, style=css_btn),
    opts,    
  ], id='left-panelDiv', style=css_lpd),

  # GRAPH
  html.Div([
#    db.Clustergram(data=df),
  ], id='graph-panelDiv', style=css_rpd)

], id='app', style={'display': 'flex'})

## 6. Javascript clientside callbacks

In [18]:
# open-close options section
app.clientside_callback(
    """
    function(largeValue1, largeValue2) {
        var x = document.getElementById("optionsDiv");
        var y = document.getElementById("options");
        var z = document.getElementById("left-panelDiv");
        if (x.style.display === "none") {
            x.style.display = "block";
            y.style.backgroundColor = "#D6F2FA";
            y.style.color = "#90B6C1";
            y.innerText = "✕";
            z.width = "25vw";
        } else {
            x.style.display = "none";
            y.style.backgroundColor = "#008CBA";
            y.style.color = "white";
            y.innerText = "≡";
            z.width = "2.1%";
        }
    }
    """,
    Output('optionsDiv', 'style'),
    Input('options', 'n_clicks'),
)



## 7. Callbacks responsive to changes in Dash widgets (options panel)

In [19]:
#_____CALLBACKS CHANGING OPTIONS_____#

# 
@app.callback(
    Output("accordion-contents", "children"),
    [Input("accordion-options", "active_item")],
)
def change_item(item):
    return f"Item selected: {item}"
  
    
# color scale modal open-close
@app.callback(Output("modal-cs", "is_open"),
             [Input("modal-cs-btn-open", "n_clicks"), Input("modal-cs-btn-close", "n_clicks"),],
             [State("modal-cs", "is_open")])
def toggle_modal(n_open, n_close, is_open):
    if n_open or n_close:
        return not is_open
    return is_open

In [20]:
#_____CALLBACKS CHANGING STORAGE_____#

# The function creates a dict of user-loaded files that can be used for plotting
@app.callback(Output('user-files-list', 'data'),
             [Input('upload-box', 'filename'), Input('upload-box', 'contents'), Input('download-btn', 'n_clicks')],
             [State('custom-url', 'value'), State('user-files-list', 'data')],
              prevent_initial_call = True)
def create_input_list(files_box, contents_box, url_clicks, url, files):
    if len(files_box):
        for num, i in enumerate(files_box):
            if i != '' and i not in files:
                content_type, content_string = contents_box[num].split(',')
                files[i] = content_string
    if len(url):
        filename = url.strip().split('/')[-1]
        if filename not in files:
            try:
                with request.urlopen(url) as f:
                    content = f.read().decode('utf-8')
                    content = base64.b64encode(content.encode('ascii')).decode('ascii')
                files[filename] = content
            except:
                pass
    return files


# Display summary of loaded inputs
@app.callback(
    [Output("settings-upload-label", "children"), Output("settings-upload-inputs", "children")],
    [Input("user-files-list", "data")])
def display_inputs_settings(loaded_files):
    if not len(loaded_files):
        return["Please load inputs using available options.", '']
    else:
        info = "The following files were loaded: "
        return [info, [generate_html_label('- '+str(i)) for i in loaded_files]]


# The function creates pandas dataframe from encoded content of selected file
@app.callback(Output('graph-data', 'data'),
             [Input('selected', 'value')],
             [State('user-files-list', 'data')],
              prevent_initial_call = True)
def create_dataframe(selected, files):
    content = files[selected]
    
#    decoded = base64.b64decode(files[i].encode('ascii')).decode('ascii')
    decoded = base64.b64decode(files[i])
    if i.endswith('.csv'):
        df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
    elif i.endswith('.xls') or i.endswith('.xlsx'):
        df = pd.read_excel(io.BytesIO(decoded))
    elif i.endswith('.json'):
        decoded = base64.b64decode(files[i].encode('ascii')).decode('ascii')
        df = pd.read_json(decoded, orient ='index')
    

In [21]:
#_____CALLBACKS CHANGING GRAPH_____#



## 8. Run local server

In [ ]:
if __name__ == "__main__":
    # Debug/Development
    app.server.run(debug=False, port=8080)
    # Production
#    from gevent.pywsgi import WSGIServer
#    http_server = WSGIServer(('', 8080), app.server)
#    http_server.serve_forever()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Dec/2022 20:56:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Dec/2022 20:56:12] "GET /static/mycss.css HTTP/1.1" 200 -
127.0.0.1 - - [19/Dec/2022 20:56:12] "GET /_dash-component-suites/dash/deps/react@16.v2_7_1m1671052907.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Dec/2022 20:56:12] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_7_1m1671052907.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Dec/2022 20:56:12] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_7_1m1671052907.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Dec/2022 20:56:12] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.v1_2_1m1671053068.min.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Dec/2022 20:56:12] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_7_1m1671052907.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [19/Dec/2022 20:56:12] "GET /_dash-component-suites/dash_bio/bundle.v1_0_2m1671052979